## GUI

Este proyecto utiliza Gradio para crear una interfaz de chat que interactúa con el modelo Llama3.2, permitiendo al usuario hacer preguntas sobre diversos temas. Las respuestas se generan a partir de documentos que el usuario envia al modelo, lo que facilita la interacción y consulta directa con el modelo

Importa las librerías necesarias

In [17]:
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.llms import Ollama

Configuramos el modelo y los embeddings

In [18]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")
llm = Ollama(model="llama3.2")

vector_store = None
retriever = None

Procesa un archivo PDF y crea un vector store basado en los embeddings.

In [19]:
def extract_data_fromPDF(pdf_file):
    global vector_store, retriever

    # Cargar el PDF
    loader = PyPDFLoader(pdf_file.name)
    documents = loader.load()

    # Dividir texto en fragmentos
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
    split_docs = text_splitter.split_documents(documents)

    # Crear embeddings para el vector store
    vector_store = Chroma.from_documents(
        documents=split_docs,
        collection_name='rag-gui-1',
        embedding=embeddings,
    )

    # Crear un retriever
    retriever = vector_store.as_retriever()

    return "PDF cargado con éxito"


Responde una pregunta utilizando el contenido del vector store o el modelo directamente.


In [20]:

def chat_answer(question):
    if retriever:
        # Recuperar contexto relevante si se ha cargado un PDF
        results = retriever.get_relevant_documents(question)
        context = "\n".join([doc.page_content for doc in results])

        # Crear el prompt
        template = """Responde en base al siguiente contexto:
        {context}

        Pregunta: {question}
        Respuesta: en español.
        """
        prompt = ChatPromptTemplate.from_template(template)
        final_prompt = prompt.format(context=context, question=question)

        response = llm(final_prompt)
    else:
        # Si no hay documentos cargados, responder directamente
        response = llm(question)

    return response

Crear la interfaz de Gradio

In [21]:
with gr.Blocks(theme=gr.themes.Glass()) as demo:
    gr.Markdown("# GUI")
    gr.Markdown("Carga un archivo PDF y realiza preguntas sobre el documento")

    with gr.Row():
        question_input = gr.Textbox(label="Pregunta", placeholder="Escribe aquí tu pregunta")
        answer_output = gr.Textbox(label="Respuesta")

    ask_button = gr.Button("Responder")
    ask_button.click(chat_answer, inputs=question_input, outputs=answer_output)

    with gr.Row():
        pdf_input = gr.File(label="Sube aquí tu archivo PDF", type="filepath")
        pdf_status = gr.Label(label="Estado del PDF")

    process_button = gr.Button("Procesar documento")
    process_button.click(extract_data_fromPDF, inputs=pdf_input, outputs=pdf_status)

demo.launch()


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


En conclusión, la interfaz de chat con Llama3.2 proporciona una manera sencilla y efectiva de obtener respuestas a preguntas sobre una variedad de temas, utilizando un modelo de lenguaje potente y accesible a través de Gradio. Esto facilita la interacción con el modelo y mejora la experiencia del usuario.